In [2]:
from __future__ import absolute_import, division, print_function
%matplotlib inline

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, MaxPooling2D, Lambda
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
#import keras.backend as K

import random
import matplotlib.pyplot as plt
import sys
import numpy as np
